# Quickstart using Python

In this tutorial you'll learn how to read GIRAFE files in Python using xarray and perform a simple computation.

## Collect Input Files

Collect daily GIRAFE files in your data directory

In [1]:
from pathlib import Path

data_dir = Path("/my/data/girafe")
filenames = data_dir.glob("PREdm*.nc")

## Create Dataset

Concatenate and merge files into an xarray dataset. Note that this does not load the data into memory until actually needed. See https://docs.xarray.dev/en/stable/user-guide/dask.html#dask-io 

In [2]:
import xarray as xr

ds = xr.open_mfdataset(filenames, concat_dim="time", combine="nested")
ds = ds.sortby("time")
ds

<xarray.Dataset>
Dimensions:                      (time: 365, nv: 2, lat: 180, lon: 360)
Coordinates:
  * time                         (time) datetime64[ns] 2022-01-01 ... 2022-12-31
  * lat                          (lat) float64 -89.5 -88.5 -87.5 ... 88.5 89.5
  * lon                          (lon) float64 -179.5 -178.5 ... 178.5 179.5
Dimensions without coordinates: nv
Data variables:
    time_bnds                    (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds                     (time, lat, nv) float64 dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds                     (time, lon, nv) float64 dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    record_status                (time) int8 dask.array<chunksize=(1,), meta=np.ndarray>
    precipitation                (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sampling_error               (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sampling_error_quality_flag  (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    num_obs_fraction             (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    num_obs_rate                 (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes: (12/32)
    title:                      Global Interpolated RAinFall Estimation (GIRAFE)
    summary:                    This file contains data from the Precipitatio...
    id:                         DOI:10.5676/EUM_SAF_CM/GIRAFE/V001
    product_version:            1.0
    creator_name:               DE/DWD
    creator_email:              contact.cmsaf@dwd.de
    ...                         ...
    platform_vocabulary:        GCMD Platforms, Version 16.5
    instrument:                 SSMIS > Special Sensor Microwave Imager/Sound...
    instrument_vocabulary:      GCMD Instruments, Version 16.5
    variable_id:                precipitation
    CMSAF_processor:            girafe-app-v1.5.0
    license:                    The CM SAF data are owned by EUMETSAT and are...

## Compute Something

Compute a timeseries of global (area-weighted) mean precipitation.

In [3]:
import numpy as np

weights = np.cos(np.deg2rad(ds["lat"]))
global_mean_precip = ds["precipitation"].weighted(weights).mean(dim=("lon", "lat"))
global_mean_precip

<xarray.DataArray 'precipitation' (time: 365)>
dask.array<truediv, shape=(365,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2022-12-31

## Plot Results

In [ ]:
global_mean_precip.plot()